In [11]:
from transformers import HubertModel
import torch
import librosa
import IPython.display as ipd
import torchaudio

In [7]:
source_path = "wavs/1_reference.wav"

In [8]:
class HubertModelWithFinalProj(HubertModel):
    def __init__(self, config):
        super().__init__(config)

        self.final_proj = torch.nn.Linear(config.hidden_size, config.classifier_proj_size)


In [9]:
contentvec_extractor = HubertModelWithFinalProj.from_pretrained("lengyue233/content-vec-best")
_ = contentvec_extractor.cuda().eval()

In [12]:
wav, sr = librosa.load(source_path)
print('Source speech')
ipd.display(ipd.Audio(wav, rate=sr))
wav = torch.FloatTensor(wav).unsqueeze(0)

Source speech


In [23]:
resample_fn = torchaudio.transforms.Resample(sr, 16000).to("cuda")
wav = wav.cuda()
mel_length = wav.shape[-1] // 256
wav = resample_fn(wav)

/home/sadjava/anaconda3/lib/python3.11/site-packages/torchaudio/functional/functional.py:1466: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  resampled = torch.nn.functional.conv1d(waveform[:, None], kernel, stride=orig_freq)


In [27]:
with torch.no_grad():
    contentvec = contentvec_extractor(wav)["last_hidden_state"]
    
contentvec = contentvec.cuda()
contentvec_length = torch.LongTensor([contentvec.shape[1]]).cuda()


/home/sadjava/anaconda3/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


In [29]:
contentvec.shape

torch.Size([1, 834, 768])